In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:

size = 1_000_000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random_int(1, 11).cast(pl.String).alias("int1"),
    pds.random_int(1, 10).cast(pl.String).alias("int2"),
    pl.col("id").cast(pl.UInt32)
)

In [ ]:
df.select(
    pl.corr("x", "y")
)

In [ ]:

df.select(
    pds.kendall_tau("x", "y")
)

In [ ]:
from scipy.stats import kendalltau

x = df["x"].to_numpy()
y = df["y"].to_numpy()

In [ ]:
%%timeit
kendalltau(x,y, nan_policy="omit")

In [ ]:
df.sort(pl.col("x").rank(method="random")).select(
    "x",
    "y",
    pl.col("y").rank(method="max").cast(pl.Float64).alias("r"),
    (-pl.col("y")).rank(method="max").cast(pl.Float64).alias("l"),
).with_columns(
    pl.col("r").diff().abs().alias("r_abs_diff"),
    (pl.col("l") * (pl.len() - pl.col("l"))).alias("l(n-l)"),
).select(
    1 - (pl.len() / 2) * (pl.col("r_abs_diff").sum() / pl.col("l(n-l)").sum())
)